<a href="https://colab.research.google.com/github/vilcagamarracf/Inv_arroz_METRIC/blob/main/files/Inv_Arroz_METRIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<br>
<img src="https://github.com/vilcagamarracf/Inv_arroz_METRIC/blob/main/figures/Github%20Cover%20-%20METRIC.png?raw=true"/>
<h1>Aplicación del modelo METRIC en cultivos de Arroz, Lambayeque</h1>
<h2> Investigación 2021 </h2>
</center>

**Information about this project**
```
FILE          : Inv-Arroz-METRIC.ipynb
AUTHOR        : Cesar Francisco Vilca Gamarra 
EMAIL         : vilcagamarracf@gmail.com 
ORGANIZATION  : UNALM 
CREATION DATE : April 29, 2021 
DEPENDENCIES  : earthengine-api, os, folium, pandas, matplotlib, numpy ...  
PURPOSE       : Download Landsat Products to estimate spatial ET
```


En esta libreta se realizará la exploración, selección y procesamiento de imágenes satelitales del producto Landsat 8 SR usando Google Earth Engine en Google Colab (Python) con el objetivo de replicar el modelo METRIC en cultivos de arroz para un área de estudio en el departamento de Lambayeque.

# Estimación de evapotranspiración: Descripción

**¿Por qué medir/estimar la evapotranspiración?**

- La evapotranspiración (ET) es un componente esencial en modelos hidrológicos y de circulación general.
- Es usado para inferir la humedad del suelo, como dato de entrada para pronósticos climáticos y de inundaciones.
- Para mejorar la eficiencia de riego a corto plazo.



**Problemática**

- Estimación de ET común:
  $$
  \text{ET} = \text{ET}_{ref} * K_c
  $$
  Dónde:
  - $\text{ET}$ : Evapotranspiración
  - $\text{ET}_{ref}$ : Evapotranspiración de referencia basada en el clima
  - $K_c$: Coeficiente de cultivo (de acuerdo al tipo y etapa de crecimiento)

- Dificultades de la estimación común:
  - Confiabilidad de usar valores idealizados de $K_c$ ya que se generaron para ciertas condiciones reales de crecimiento y vegetación.
  - Identificación de etapas vegetativas y condiciones de crecimiento para comparación con valores de $K_c$.
  - Predicción correcta de fechas según etapa de crecimiento para cultivos de grandes extensiones.


**Sensoramiento Remoto como una solución**

Los satélites son capaces de obtener información espacial de evapotranspiración de numerosas extensiones a partir de técnicas de balance de energía.   

  > Estimación de la evapotranspiración de los cultivos bajo riego mediante imágenes de sensores remotos (tipo de sensor y bandas del espectro electromagnético)

**Métodos de estimación de ETP con Sensoramiento Remoto**

Kalma et al. ([2008](https://link.springer.com/article/10.1007%2Fs10712-008-9037-z)) reúne las **metodologías existentes** para estimar la evapotranspiración de los cultivos usando Sensoramiento Remoto, las cuales son:
  - **Balance Energético de la Superficie**
  - Métodos estadísticos que utilizan diferencias entre la temperatura de la superficie y el aire
  - Correlaciones simplificadas o relaciones entre extremos de temperatura superficial de una imagen y puntos finales del ET anticipado
  - ET relativa basada en la vegetación que se multiplica por una ET de referencia basada en el tiempo

Más información en *Operational Remote Sensing of ET and Challenges* [(2011)](https://www.intechopen.com/books/evapotranspiration-remote-sensing-and-modeling/operational-remote-sensing-of-et-and-challenges).



**Balance Energético de la Superficie**

Se subdivide en:
-  Balance energético completo para la imagen satelital: 
    \begin{equation}\lambda{E}=R_n - G -H \end{equation}
     dónde:
     - $\lambda{E}$ es la densidad latente del flujo térmico, representando la energía "consumida" por la evaporación del agua, 
     - $R_n$ es la densidad neta del flujo de radiación, 
     - $G$ es la densidad del flujo de calor del suelo, y 
     - $H$ es la densidad sensible del flujo de calor al aire.
- Índice de estrés hídrico basado en la temperatura superficial y las cantidades de vegetación. 
- Aplicación de un Modelo Continuo de Superficie Terrestre (MSL) que se inicializa parcialmente y avanzado, en el tiempo, utilizando imágenes satelitales

Todas las metodologías anteriormente mencionadas solamente trabajan sobre imágenes disponibles y dependiendo de la revisita del mismo, quedan vacíos de información entre imágenes.

**Modelo METRIC** 

> METRIC uses TIR-Multispectral satellite images (e.g., from Landsat 7 ETM+ and 8 OLI) and ground-based meteorological data to estimate pixel-based daily $ET_a$ at 30 × 30 m spatial resolution via a surface energy balance equation at the time of the satellite overpass 

<center>
<img src= 'https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fimage.slideserve.com%2F327001%2Fhow-metric-works6-l.jpg&f=1&nofb=1' width=50%/>
</center>

Datos principales:
- Imágenes termales satelitales de onda corta y larga (Landsat 7 ETM+ y Landsat 8 OLI/TIRS)
- Un modelo digital de elevación (DEM)
- Datos de medición de campo de la zona de interés o cercana.

A continuación procederemos con la parte práctica usando la API de Python de Earth Engine con Colab. 

# METRIC con Earth Engine en Colab

### Inicio

- Ingreso de credenciales de Earth Engine 
- Importar librerías y funciones 
- ROI Lambayeque

Primero ingresamos nuestros datos de Google Earth Engine (necesario una cuenta registrada).

In [ ]:
# # Ingresa tus credenciales de Google Earth Engine para comenzar
# import ee
# ee.Authenticate()

# # Inicio
# ee.Initialize()

# # Mensaje de login
# print('\nYa estás logeado.')
# print('GEE Version:', ee.__version__)

In [ ]:
#@markdown *Login: Credenciales Google Earth Engine*
import os 
credential = '{"refresh_token":"1//09lhqedCNpNDsCgYIARAAGAkSNwF-L9IraEP8FyJma46WSiEHhOMsZqSoTPJsi3Lar0C4nZUwj2QCZ_i66-McVzy4NmMalQL17KM"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)
import ee
ee.Initialize()
print('Ya estás logeado.')
print('GEE Version:',ee.__version__)

Ya estás logeado.
GEE Version: 0.1.290


Cargar funciones extra

In [ ]:
# Librerías
import pandas as pd
from ipywidgets import interact, fixed

In [ ]:
# Mejorar visibilidad de tablas en Colab
%load_ext google.colab.data_table

In [ ]:
# @markdown Función `ver_rangos(img_col, date_start, date_end)`
# @markdown - Presenta los rangos de fecha ingresada y existente en Earth Engine
# ver_rangos ----------------------------------------

def ver_rangos(img_col, date_start, date_end):
  """
  Devuelve los rangos de fechas existente en un rango inicial escrito manualmente como input.
  Pasos:
  1. `Reducer.minMax()` devuelve un valor min y max
  2. `icol.reduceColumns()` devuelve un diccionario con el min-max
  more info: 
  https://developers.google.com/earth-engine/apidocs/ee-imagecollection-reducecolumns?hl=en
  """
  
  rango = img_col.reduceColumns(ee.Reducer.minMax(), ["system:time_start"]) 
  # Retorna un ee.dictionary: {'max': 1608132402761, 'min': 1545060401000}

  # Obtención de la fecha min y max del image collection en formato ISO standard 8601
  # Javascript trabaja las fechas con milisegundos (se deja así)
  # Python     trabaja las fechas con segundos (por eso /1000)
  init_date = ee.Date(rango.get('min')).getInfo()['value']/1000.
  last_date = ee.Date(rango.get('max')).getInfo()['value']/1000.

  # Dar formato a las fechas  
  from datetime import datetime as dt
  init_date_f = dt.utcfromtimestamp(init_date).strftime('%Y-%m-%d') # %H:%M:%S
  last_date_f = dt.utcfromtimestamp(last_date).strftime('%Y-%m-%d') # %H:%M:%S
  
  # Obtenga el rango de fechas de las imágenes en la colección.
  range_date = [date_start, date_end]
  print('Total imágenes: {}'.format(img_col.size().getInfo()))
  print('Rango Temporal Ingresado : {} - {}'.format(range_date[0], range_date[1]))
  print('Rango Temporal Real      : {} - {}'.format(init_date_f, last_date_f))

Ahora instalamos `geemap` en Colab para la implementación de una interfaz interactiva donde podamos visualizar objetos vectoriales y raster. 

In [ ]:
%%capture
!pip install -U geemap
# !pip install geemap==0.10.1

In [ ]:
# Interfaz interactiva con geemap 
import geemap
Map = geemap.Map(basemap='HYBRID', center=(-9.125, -74.396), zoom=5)
# Map = geemap.Map(basemap='OpenStreetMap.Mapnik', center=(-9.125, -74.396), zoom=5) # Para visualización minimalista
Map

Map(center=[-9.125, -74.396], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

Con geemap podemos obtener visualizaciones como la siguiente: Distrito de Chongoyape, Chiclayo, Lambayeque.

In [ ]:
shpPeru = ee.FeatureCollection('users/CesarVilca/Departamentos_Peru') # Asset de EarthEngine
shpDep = shpPeru.filter(ee.Filter.eq('NOMBDEP', 'LAMBAYEQUE'))

shpPeru_distritos = ee.FeatureCollection('users/CesarVilca/Distritos_Peru_WGS84') # Asset de EarthEngine
shpChongoyape = shpPeru_distritos.filter(ee.Filter.eq('NOMBDIST', 'CHONGOYAPE'))

# roiPeru = shpPeru.geometry()
# roiDep  = shpDep.geometry()
roiChongoyape = shpChongoyape.geometry()

chongoyape = ee.Geometry.Point([-79.389, -6.639])
oyotun     = ee.Geometry.Point([-79.309, -6.846])
centroide  = ee.Geometry.Point([-79.823, -6.351])

In [ ]:
# @markdown Visualización con `geemap`: Distrito de Chongoyape
# Visualización
roiPeruDraw = shpPeru.draw(
    color = "D8EED0", strokeWidth = 1, pointRadius = 2) 
roiDepDraw  = shpDep.draw(
    color = "00FF00", strokeWidth = 2, pointRadius = 2) 
roiChongoyapeDraw  = shpChongoyape.draw(
    color = "00FFFF", strokeWidth = 2, pointRadius = 2) 

# Visualización: geemap
Map = geemap.Map(basemap='HYBRID', layer_ctrl=True) # OpenStreetMap.Mapnik
Map.centerObject(shpChongoyape, 9) # Map.setCenter(lon, lat, zoom)
Map.addLayer(roiPeruDraw, {}, name = 'Perú')
Map.addLayer(roiDepDraw, {}, name = 'Lambayeque')
Map.addLayer(roiChongoyapeDraw, {}, name = 'Chongoyape')

Map

Map(center=[-6.626857365812768, -79.46457298069963], controls=(WidgetControl(options=['position', 'transparent…

## Exploración y selección de imágenes Landsat 8

### Constelación Landsat

**Landsat** 

Programa conjunto del USGS y la NASA que ha estado observando la Tierra continuamente desde 1972 hasta la actualidad. Hoy en día, los satélites Landsat obtienen imágenes de toda la superficie de la Tierra a una resolución de 30 metros aproximadamente una vez cada dos semanas, incluidos datos multiespectrales y térmicos. El USGS produce datos en 3 categorías para cada satélite (Nivel 1, Nivel 2 y RT).


**Landsat Collections**

Landsat se divide en dos colecciones:

- ***Landsat Collection 1***: Inició en 2016 y contiene toda la data de Level-1 adquirida desde 1972 hasta el presente de Landsat 1-8.
  - En Earth Engine: [USGS Landsat 8 Surface Reflectance Tier 1 [deprecated] ](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR)
- ***Landsat Collection 2***: Inició a mediados del año 2021 y representa un hito en la historia de Landsat ya que realiza mejoras sustanciales al procesamiento de las imágenes antes adquiridas, dando como resultado un mayor aprovechamiento de los avances recientes en el procesamiento de datos, el desarrollo de algoritmos y las capacidades de acceso y distribución de datos.
  - En Earth Engine: [USGS Landsat 8 Level 2, Collection 2, Tier 1](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

**Nota**: A partir del primero de enero del 2022, todas las imágenes adquiridas y procesadas serán almacenadas solamente en el inventario *Landsat Collection 2*. [Fuente: USGS](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collections?qt-science_support_page_related_con=2#qt-science_support_page_related_con)


**Landsat 8 en Earth Engine**

A continuación veremos las propiedades de las bandas del producto Landsat 8 Surface Reflectance Tier 1 en el dataset de Earth Engine. 

\

|Name	|Units|	Scale|	Wavelength|	Description|
|:-:|:-:|-|-|-|
|B1		|-|0.0001	|0.435-0.451 μm	|Band 1 (ultra blue) surface reflectance|
|B2		|-|0.0001	|0.452-0.512 μm	|Band 2 (blue) surface reflectance|
|B3		|-|0.0001	|0.533-0.590 μm	|Band 3 (green) surface reflectance|
|B4		|-|0.0001	|0.636-0.673 μm	|Band 4 (red) surface reflectance|
|B5		|-|0.0001	|0.851-0.879 μm	|Band 5 (near infrared) surface reflectance|
|B6		|-|0.0001	|1.566-1.651 μm	|Band 6 (shortwave infrared 1) surface reflectance|
|B7		|-|0.0001	|2.107-2.294 μm	|Band 7 (shortwave infrared 2) surface reflectance|
|B10	|Kelvin	|0.1	|10.60-11.19 μm|Band 10 brightness temperature.|
|B11	|Kelvin	|0.1	|11.50-12.51 μm|Band 11 brightness temperature.|

\

**Nota**:
Las bandas B10 y B11, aunque se recopilaron originalmente con una resolución de 100 m / píxel, se han vuelto a muestrear mediante convolución cúbica a 30 m.

### Exploración de imágenes del dataset Landsat 8 Surface Reflectance

Se realizará un filtrado al dataset con el fin de reducir la cantidad de imágenes a las que abarquen nuestra área de estudio. Es posible el filtrado mediante: 
- Un punto: coordenadas como longitud y latitud de un lugar exacto
- Multipuntos: archivo shapefile como conjunto de coordenadas
- Propiedades: Path y Row
  - **Nota**: Para ubicar el Path y Row de la zona de estudio de forma interactiva, usar el siguiente enlace: [GEO GPS PERÚ: Ubica el PATH y ROW de tu distrito para imágenes LANDSAT](https://www.geogpsperu.com/2016/02/ubica-el-path-y-row-de-tu-distrito-para.html)


In [ ]:
# @markdown Función `filtrado_con_roi(ID_snippet_name, year, roi)`
# @markdown - Permite ver las imágenes que abarca el `roi` para el año e ID dado
# @markdown - Retorna: `ee.Image`
def filtrado_con_roi(ID_snippet_name, year, roi):
  '''
  Argumentos:
  - ID_snippet_name: str
  - year: int
  - roi: ee.Geometry, ee.FeatureCollection
  Retorna: 
  - ee.Image
  '''  
  date1 = str(year)
  date2 = str(year+1)

  icol_clean = (
    ee.ImageCollection(ID_snippet_name)\
    .filterDate(date1, date2)\
    .filterBounds(roi)
    )
  
  if ID_snippet_name == 'LANDSAT/LC08/C01/T1_SR':
    cloud_property = 'CLOUD_COVER'
    layer_title = 'Imágenes Landsat 8 SR'
  
  elif ID_snippet_name == 'COPERNICUS/S2_SR':
    cloud_property = 'CLOUDY_PIXEL_PERCENTAGE'
    layer_title = 'Imágenes Sentinel-2 SR'

  icol_clean_sorted = icol_clean.sort(cloud_property)
  
  mejor_imagen = icol_clean_sorted.first().date().format('YYYY-MM-dd')
  fecha1 = ee.Date(mejor_imagen)
  fecha2 = fecha1.advance(16, 'day') # ee.Date().advance()

  # Rango de Fechas
  ver_rangos(icol_clean, date1, date2)
  print(f"Fecha de mejor toma: {mejor_imagen.getInfo()}")

  # 2do Fitrado: Visualización de imágenes con mejor toma
  icol_best = (
      ee.ImageCollection(ID_snippet_name)\
      .filterDate(fecha1, fecha2)\
      .filterBounds(roi)\
      # .sort(cloud_property)
  )

  imagenes = icol_best.mosaic().multiply(0.0001)
  
  return icol_clean, imagenes 

In [ ]:
# Filtrado de imágenes Landsat 8 para el Distrito de Chongoyape
icol_L8, imagenes_L8 = filtrado_con_roi("LANDSAT/LC08/C01/T1_SR", 2021, chongoyape)

Total imágenes: 34
Rango Temporal Ingresado : 2021 - 2022
Rango Temporal Real      : 2021-01-02 - 2021-10-17
Fecha de mejor toma: 2021-02-03


In [ ]:
# Visualizacion
dict_imagenes = {'min':0.0, 'max':0.3, 'bands': ['B4','B3','B2']}

Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)
Map.centerObject(chongoyape, 8)
Map.addLayer(imagenes_L8, dict_imagenes, name = 'Imágenes Landsat')
Map.addLayer(roiDepDraw, {}, 'Lambayeque')
Map.addLayer(oyotun, {}, 'Oyotun')
Map.addLayer(roiChongoyapeDraw, {}, name = 'Chongoyape')
Map.addLayer(chongoyape, {}, 'Distrito Chongoyape')

Map

Map(center=[-6.639, -79.389], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

### Selección de imágenes: Landsat 8

- Elaboración de una tabla con la información de propiedades para el año 2021
- Visualización de imágenes satelitales


#### Generar tabla de propiedades

Se va a generar una tabla con información sobre imágenes para el año 2020 mediante la librería `pandas`.

Para más información sobre las nombres de propiedades del ee.ImageColection: 
  - https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR#image-properties

In [ ]:
# @markdown Función `generar_reporte(icol)`

def generar_reporte(icol):
  """Generar reportes para ee.ImageCollection 's
  
  A partir de un ee.ImageCollection devolver una tabla 
  con información de ID, Fechas (Año, Mes, Día, Día Juliano) y 
  propiedades como Ángulo Zenital y Porcentaje de Nubosidad.

  Argumentos:
  - icol : ee.ImageCollection
  Retorna:
  - pandas.Dataframe
  """
  ## Generando campos para la tabla
  ID_snippet_name = icol.getInfo()['id']

  # Lista con fechas (en milisegundos)
  lista_fechas = icol.aggregate_array('system:time_start').getInfo() # Fechas
  # Lista con ID's
  imgCol_ids = [f'{ID_snippet_name}/{i}' for i in icol.aggregate_array("system:index").getInfo()] # ID

  # Armando dataframe
  df = pd.DataFrame(lista_fechas, columns = ['millis'])

  df["ID GEE"] = imgCol_ids
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Fecha Precisa'] = pd.DatetimeIndex(df['Timestamp']) # Con hora
  # df['Fecha Corta'] = pd.DatetimeIndex(df['Timestamp']).date
  df['Año'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Mes'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Día'] = pd.DatetimeIndex(df['Timestamp']).day
  # df['Hora'] = pd.DatetimeIndex(df['Timestamp']).hour
  df['Día Juliano'] = pd.DatetimeIndex(df['Timestamp']).dayofyear

  # Agregando propiedades
  if ID_snippet_name == 'LANDSAT/LC08/C01/T1_SR':
    df["Zenith Angle"] = icol.aggregate_array('SOLAR_ZENITH_ANGLE').getInfo()
    df["Porcentaje Nubes (%)"] = icol.aggregate_array('CLOUD_COVER').getInfo()
  
  elif ID_snippet_name == 'COPERNICUS/S2_SR':
    df["Zenith Angle"] = icol.aggregate_array('MEAN_SOLAR_ZENITH_ANGLE').getInfo()
    df["Porcentaje Nubes (%)"] = icol.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo()
  
  elif ID_snippet_name == 'LANDSAT/LC08/C02/T1_L2':
    df["SUN_ELEVATION"] = icol.aggregate_array('SUN_ELEVATION').getInfo()
    df["CLOUD_COVER"] = icol.aggregate_array('CLOUD_COVER').getInfo()
    
  df = df.drop(columns=['millis', 'Timestamp'])
  # df.to_csv('datos_2020_L8_SR.csv')
  
  return df

In [ ]:
# Generar tabla con propiedades de un ee.ImageCollection
reporteL8 = generar_reporte(icol_L8)
reporteL8

,ID GEE,Fecha Precisa,Año,Mes,Día,Día Juliano,Zenith Angle,Porcentaje Nubes (%)
0,LANDSAT/LC08/C01/T1_SR/LC08_010064_20210102,2021-01-02 15:28:10.176,2021,1,2,2,32.518929,52.06
1,LANDSAT/LC08/C01/T1_SR/LC08_010064_20210203,2021-02-03 15:28:01.323,2021,2,3,34,32.278900,45.94
2,LANDSAT/LC08/C01/T1_SR/LC08_010064_20210307,2021-03-07 15:27:49.609,2021,3,7,66,30.337090,52.46
3,LANDSAT/LC08/C01/T1_SR/LC08_010064_20210323,2021-03-23 15:27:42.401,2021,3,23,82,30.182537,20.74
4,LANDSAT/LC08/C01/T1_SR/LC08_010064_20210408,2021-04-08 15:27:38.007,2021,4,8,98,31.127533,51.72
5,LANDSAT/LC08/C01/T1_SR/LC08_010064_20210424,2021-04-24 15:27:30.566,2021,4,24,114,33.085968,31.38
6,LANDSAT/LC08/C01/T1_SR/LC08_010064_20210510,2021-05-10 15:27:25.850,2021,5,10,130,35.561172,32.32
7,LANDSAT/LC08/C01/T1_SR/LC08_010064_20210526,2021-05-26 15:27:35.897,2021,5,26,146,37.914558,46.51
8,LANDSAT/LC08/C01/T1_SR/LC08_010064_20210611,2021-06-11 15:27:42.933,2021,6,11,162,39.630791,87.08
9,LANDSAT/LC08/C01/T1_SR/LC08_010064_20210627,2021-06-27 15:27:47.134,2021,6,27,178,40.297325,51.09


#### Visualización

Datasets:
- USGS Landsat 8 Surface Reflectance Tier 1
- USGS Landsat 8 Level 2, Collection 2, Tier 1

###### USGS Landsat 8 Surface Reflectance Tier 1 [deprecated] - [Dataset](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR)

Con ayuda de [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#) es posible realizar visualizaciones interactivas con HTML siendo el indicado para ello el módulo [`interact`](https://colab.research.google.com/github/jupyter-widgets/ipywidgets/blob/master/docs/source/examples/Using%20Interact.ipynb).

1. A partir del reporte generado obtenemos los ID's

2. Desarrollamos la función `visualizar_mes(mes)` que permita la visualización de imágenes de forma mensual.


In [ ]:
# @markdown Función `visualizar_mes_L8_SR(mes)`

def visualizar_mes_L8_SR(mes, df, roi):
  
  # Del dataframe obtenemos las imagenes de acuerdo al mes que queramos
  lista_imagenes = df[df['Mes'] == mes]['ID GEE'].tolist()

  # Parametros de visualización RGB
  vis_rgb = {'min': 0.0, 'max': 0.3, 'gamma':1.4, 'bands': ['B4', 'B3', 'B2']}
  
  # geemap para la visualización
  Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)

  Map.centerObject(roi, 12)  # Map.setCenter(-79.809, -6.746, 9)
  for i in lista_imagenes:
    Map.addLayer(
        ee.Image(i).multiply(0.0001), vis_rgb, f'Imagen {mes}/{i[-2:]}'
        ) # .clip(roiDep)

  return Map

3. Corremos `interact` con la función anteriormente desarrollada. Para visualizar las imágenes solo es necesario cambiar el valor de `mes` y escoger entre los valores del 1 al 12.

In [ ]:
mes = reporteL8['Mes'].unique().tolist()
interact(visualizar_mes_L8_SR, mes=mes, df=fixed(reporteL8), roi=fixed(chongoyape));

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=1), Output())…

###### USGS Landsat 8 Level 2, Collection 2, Tier 1 - [Dataset](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

1. Generamos el reporte y así obtener los ID's


In [ ]:
ID_snippet_name = "LANDSAT/LC08/C02/T1_L2"
path = 10
row = 65
year = 2021

## Filtrado de ee.imageCollection
icol_sr = (
    ee.ImageCollection(ID_snippet_name)
    .filterDate(str(year), str(year+1))
    .filterMetadata('WRS_PATH', 'equals', path)
    .filterMetadata('WRS_ROW', 'equals', row)
)

In [ ]:
reporteL8_C2 = generar_reporte(icol_sr)
reporteL8_C2

,ID GEE,Fecha Precisa,Año,Mes,Día,Día Juliano,SUN_ELEVATION,CLOUD_COVER
0,LANDSAT/LC08/C02/T1_L2/LC08_010065_20210102,2021-01-02 15:28:34.101,2021,1,2,2,58.105871,40.75
1,LANDSAT/LC08/C02/T1_L2/LC08_010065_20210203,2021-02-03 15:28:25.244,2021,2,3,34,58.032693,2.66
2,LANDSAT/LC08/C02/T1_L2/LC08_010065_20210307,2021-03-07 15:28:13.534,2021,3,7,66,59.425698,35.54
3,LANDSAT/LC08/C02/T1_L2/LC08_010065_20210323,2021-03-23 15:28:06.322,2021,3,23,82,59.270700,11.74
4,LANDSAT/LC08/C02/T1_L2/LC08_010065_20210408,2021-04-08 15:28:01.927,2021,4,8,98,58.055825,48.35
5,LANDSAT/LC08/C02/T1_L2/LC08_010065_20210424,2021-04-24 15:27:54.495,2021,4,24,114,55.901255,28.81
6,LANDSAT/LC08/C02/T1_L2/LC08_010065_20210510,2021-05-10 15:27:49.775,2021,5,10,130,53.304948,9.97
7,LANDSAT/LC08/C02/T1_L2/LC08_010065_20210526,2021-05-26 15:27:59.826,2021,5,26,146,50.888219,88.96
8,LANDSAT/LC08/C02/T1_L2/LC08_010065_20210611,2021-06-11 15:28:06.858,2021,6,11,162,49.150305,97.46
9,LANDSAT/LC08/C02/T1_L2/LC08_010065_20210627,2021-06-27 15:28:11.059,2021,6,27,178,48.492596,88.29


In [ ]:
# @markdown Visualizar imágenes de Landsat8 L2
def visualizar_mes_L8_L2(mes, df, roi):
  
  # Del dataframe obtenemos las imagenes de acuerdo al mes que queramos
  lista_imagenes = df[df['Mes'] == mes]['ID GEE'].tolist()

  # Parametros de visualización RGB, 'gamma':1.4
  vis_rgb = {'min': 0.0, 'max': 0.3, 'bands': ['SR_B4', 'SR_B3', 'SR_B2']}
  
  # geemap para la visualización
  Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)
  Map.centerObject(roi, 12)  # Map.setCenter(-79.809, -6.746, 9)
  for i in lista_imagenes:
    Map.addLayer(
        ee.Image(i).multiply(0.0000275).add(-0.2), vis_rgb, f'Imagen {i[-2:]}')

  return Map

In [ ]:
mes = reporteL8_C2['Mes'].unique().tolist()
interact(visualizar_mes_L8_L2, mes=mes, df=fixed(reporteL8_C2), roi=fixed(chongoyape));

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=1), Outpu…

## Exploración y selección de imágenes Sentinel-2 MSI: MultiSpectral Instrument, Level-2A

Sentinel-2 (S2) is a wide-swath, high-resolution, multispectral imaging mission with a global 5-day revisit frequency. The S2 Multispectral Instrument (MSI) samples 13 spectral bands: visible and NIR at 10 meters, red edge and SWIR at 20 meters, and atmospheric bands at 60 meters spatial resolution. It provides data suitable for assessing state and change of vegetation, soil, and water cover.

### Exploración de Imágenes

In [ ]:
# Filtrado de imágenes Sentinel-2 para el Distrito de Chongoyape
icol_S2, imagenes_S2 = filtrado_con_roi('COPERNICUS/S2_SR', 2021, chongoyape)

Total imágenes: 69
Rango Temporal Ingresado : 2021 - 2022
Rango Temporal Real      : 2021-01-01 - 2021-12-02
Fecha de mejor toma: 2021-05-11


In [ ]:
# Visualizacion
dict_imagenes = {'min':0.0, 'max':0.3, 'bands': ['B4','B3','B2']}

Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)
Map.centerObject(chongoyape, 8)
Map.addLayer(imagenes_S2, dict_imagenes, name = 'Imágenes Landsat')
Map.addLayer(roiDepDraw, {}, 'Lambayeque')
Map.addLayer(oyotun, {}, 'Oyotun')
Map.addLayer(roiChongoyapeDraw, {}, name = 'Chongoyape')
Map.addLayer(chongoyape, {}, 'Distrito Chongoyape')

Map

Map(center=[-6.639, -79.389], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

### Selección de imágenes: Sentinel-2

- Elaboración de una tabla con la información de propiedades para el año 2021
- Visualización de imágenes satelitales


#### Generar tabla de propiedades

Se va a generar una tabla con información sobre imágenes para el año 2021 mediante la librería `pandas`.

In [ ]:
reporteS2 = generar_reporte(icol_S2)
reporteS2

,ID GEE,Fecha Precisa,Año,Mes,Día,Día Juliano,Zenith Angle,Porcentaje Nubes (%)
0,COPERNICUS/S2_SR/20210101T153619_20210101T1544...,2021-01-01 15:45:05.914,2021,1,1,1,28.362474,60.175373
1,COPERNICUS/S2_SR/20210106T153621_20210106T1540...,2021-01-06 15:45:08.039,2021,1,6,6,28.540159,96.552039
2,COPERNICUS/S2_SR/20210111T153619_20210111T1537...,2021-01-11 15:45:06.966,2021,1,11,11,28.625081,14.484162
3,COPERNICUS/S2_SR/20210116T153621_20210116T1540...,2021-01-16 15:45:08.324,2021,1,16,16,28.601828,92.622773
4,COPERNICUS/S2_SR/20210121T153619_20210121T1537...,2021-01-21 15:45:07.362,2021,1,21,21,28.493682,27.849255
...,...,...,...,...,...,...,...,...
64,COPERNICUS/S2_SR/20211112T153621_20211112T1540...,2021-11-12 15:45:09.353,2021,11,12,316,21.862431,9.316200
65,COPERNICUS/S2_SR/20211117T153619_20211117T1537...,2021-11-17 15:45:05.511,2021,11,17,321,22.667283,12.258994
66,COPERNICUS/S2_SR/20211122T153621_20211122T1540...,2021-11-22 15:45:07.596,2021,11,22,326,23.482486,31.135298
67,COPERNICUS/S2_SR/20211127T153619_20211127T1537...,2021-11-27 15:45:03.072,2021,11,27,331,24.320940,15.195945


<img src='https://www.aulafacil.com/uploads/cursos/5922/20208_angulos-y-altura-en-la-trayectoria-del-sol.es.jpg' width='50%'/>

Para más información sobre las nombres de propiedades del ee.ImageColection: 
- Sentinel-2 MSI: MultiSpectral Instrument, Level-2A/[Image Properties](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#image-properties)

#### Visualización

Datasets:
- [Sentinel-2 MSI: MultiSpectral Instrument, Level-2A](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)

In [ ]:
# @markdown Función `visualizar_mes_S2_SR(mes)`

def visualizar_mes_S2_SR(mes, df):
  
  # Del dataframe obtenemos las imagenes de acuerdo al mes que queramos
  lista_imagenes = df[df['Mes'] == mes]['ID GEE'].tolist()

  # Parametros de visualización RGB
  vis_rgb = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']} # 'gamma':1.4
  
  # geemap para la visualización
  Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)
  Map.centerObject(chongoyape, 12)  # Map.setCenter(-79.809, -6.746, 9)

  # Agregar imágenes por mes
  for i in lista_imagenes:
    Map.addLayer(ee.Image(i).multiply(0.0001), vis_rgb, f'Imagen {i[21:23]}/{i[23:25]}') # {i} .clip(roiDep)
 
  # Visualizar al último el distrito de Chongoyape
  Map.addLayer(chongoyape, {'color':'00FF00'}, 'Chongoyape') # roiChongoyapeDraw
  return Map

In [ ]:
mes = reporteS2['Mes'].unique().tolist()
interact(visualizar_mes_S2_SR, mes=mes, df=fixed(reporteS2));

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=1), O…

## Balance de energía superficial 

- Procesamiento de imágenes satelitales de acuerdo a la metodología establecida en el modelo METRIC.

### Descripción

**Metodología**

Se ha seguido la metodología empleada en las siguiente investigación:
- [2007: Satellite-Based Energy Balance for Mapping Evapotranspiration with Internalized Calibration (METRIC) — Model](http://cursosihlla.bdh.org.ar/ET_UNCu_Cursos/Curso_2019/1_Lectura_recomendada/2007_Allen_METRIC_Satellite-Based_Energy_Balance_for_Mapping_Evapotr.pdf)

Además de la revisión de tesis con el fin de obtener mayor información sobre la replicación del modelo:
- [2019: Estimación de la evapotranspiración de cultivo de maíz bajo riego mediante percepción remota](http://repositorio.imta.mx/handle/20.500.12013/2065)
- [2018: Estimación de la evapotranspiración en los cultivos alrededor del observatorio de Huancayo mediante sensoramiento remoto](https://repositorio.igp.gob.pe/handle/20.500.12816/4631)

Para mayor información del primer artículo e investigaciones relacionadas:
- [Connected Papers: METRIC Model and further research](https://www.connectedpapers.com/main/817edad756d41499da26498e71c85afaa884383a/SatelliteBased-Energy-Balance-for-Mapping-Evapotranspiration-with-Internalized-Calibration-METRICModel/graph)



**Base teórica y computacional: Ecuación del balance de energía**

\

$$
LE = R_n - G - H
$$

Dónde: 
- $LE$ : Energía latente consumida por ET $[W/m²]$
- $R_n$ : Radiación neta (suma de todas las radiaciones de onda corta y larga, sean entrantes o salientes a la superficie) $[W/m²]$
- $G$ : Flujo de calor sensible conducido al suelo $[W/m²]$
- $H$ : Flujo de calor sensible convectado al aire $[W/m²]$

\

<center>
<img src= 'https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.researchgate.net%2Fprofile%2FBiswajeet_Pradhan%2Fpublication%2F301679158%2Ffigure%2Ffig1%2FAS%3A412214499528704%401475291003507%2FSurface-Energy-Balance-12.jpg&f=1&nofb=1' width=50%/>
</center>

**¿Similitud con el Modelo SEBAL?**

El modelo ***SEBAL*** se basa en la ecuación del balance de energía. 

***METRIC*** en comparación realiza una calibración interna enfocada en el parámetro *H* con el fil de absorber la mayor cantidad de errores de estimación y sesgos.  

En METRIC:
- $R_n$ is computed from the satellite-measured narrow-band reflectance and surface temperature
- $G$ is estimated from $R_n$, surface temperature, and vegetation indices
- and $H$ is estimated from surface temperature ranges, surface roughness,
and wind speed using buoyancy corrections.

### Radiación Neta / Net Radiation $R_n$

Morse et al. 2000:

$$
R_n = (1- \alpha)R_{S\downarrow} + (R_{L\downarrow} - R_{L\uparrow}) - (1- \epsilon_0)R_{L\downarrow}
$$

Dónde: 
- $R_n$ : Flujo de radiación neta $[W/m^2]$
- $\alpha$ : Albedo de superficie
- $R_{S\downarrow}$ : Radiación de onda corta entrante $[W/m^2]$
- $R_{L\downarrow}$ : Radiación de onda larga entrante $[W/m^2]$
- $R_{L\uparrow}$ : Radiación de onda larga saliente $[W/m^2]$
- $\epsilon_0$ : Emisividad del ancho de banda en la superficie / broad-band surface thermal emissivity

The $(1- \epsilon_0)R_{L\downarrow}$ term represents the fraction of incoming long-wave
radiation reflected from the surface.

#### Albedo / Surface Albedo $\alpha_{s}$

##### Teoría

Mayor información: 
- Tasumi, M., Allen, R. G., & Trezza, R. (2008). At-surface reflectance and albedo from satellite for operational calculation of land surface energy balance. Journal of hydrologic engineering, 13(2), 51-63. [Ver pdf online](https://www.academia.edu/download/51734968/At-Surface_Reflectance_and_Albedo_from_S20170210-28869-3y0c0b.pdf)
- Silva, B. B. da, Braga, A. C., Braga, C. C., Oliveira, L. M. M. de, Montenegro, S. M. G. L., & Barbosa Junior, B. (2016). Procedures for calculation of the albedo with OLI-Landsat 8 images: Application to the Brazilian semi-arid. Revista Brasileira de Engenharia Agrícola e Ambiental, 20, 3-8. https://doi.org/10.1590/1807-1929/agriambi.v20n1p3-8
- Liang, S. (2001). Narrowband to broadband conversions of land surface albedo I: Algorithms. Remote Sensing of Environment, 76(2), 213-238. https://doi.org/10.1016/S0034-4257(00)00205-4



[The ASCE standard reference evapotranspiration equation 2005](https://www.mesonet.org/images/site/ASCE_Evapotranspiration_Formula.pdf)

The calculation of ETsz **uses the constant value of 0.23 for albedo for daily and hourly periods**. It is recognized that albedo varies somewhat with time of day and 
with time of season and latitude due to change in sun angle. However, because the
solar intensity is less during these periods, the error introduced in fixing albedo at
0.23 is relatively small (Allen et al., 1994b). Users may elect to use a different 
prediction for albedo, however, it is essential to ascertain the validity and accuracy of 
an alternative method using good measurements of incoming and reflected solar 
radiation.

El albedo es **la relación entre la radiación solar reflejada y la radiación solar incidente en la superficie**. Un método para obtener el valor del albedo se describe en el trabajo de Morse, Allen, y Kramber (2000), mediante la siguiente ecuación la cual está en función de la elevación, y diversas condiciones expresadas en las siguientes ecuaciones.

$$
\alpha = \frac{\alpha_{toa}-\alpha_{pr}}{\tau_{sw}^2} 
$$

Dónde:
- $\alpha$ : Albedo de la superficie / Surface albedo
- $\alpha_{toa}$ : Albedo en el límite superior de la atmósfera / Albedo at the top of the atmosphere (planetary)
- $\alpha_{pr}$ : Albedo de trayectoria, igual a 0.03 (Bastiaanssen, 2000) / Albedo atmosférico ✔️
- $\tau_{sw}$ : Transmitancia en un sentido con condiciones de claridad $[W/m^2K]$ / Transmisividad atmosférica
- $z$ : Elevación sobre el nivel del mar $[m]$, del modelo digital de elevaciones

**Transmitancias**

Para estimar $\tau_{sw}$:

$$
\tau_{sw} = \tau_B + \tau_D
$$

Donde:
- $\tau_B$ es el índice de transmisividad de radiación directa y 
- $\tau_D$ es índice de transmisividad de radiación difusa

La ecuación utilizada para el cálculo de $\tau_B$ es:

$$
\tau_B = 0.98 * \exp \left[ \frac{-0.00146*P_{air}}{K_t * \cos\theta} - 0.075 * \left( \frac {W}{\cos\theta}\right)^{0.4} \right]
$$

$P_{air}$, $W$ , $K_t$ y $\cos{θ}$ fueron definidas en las ecuaciones anteriores:

$$
P_{air} = 101.1 \left( \frac {293 - 0.0067 * z}{293}\right)^{5.26}
$$

Donde: $z$ es la elevacion media de la imagen, respecto al nivel del mar. El agua precipitable se calculo mediante la ecuación:

$$
W = 0,14 ∗ e_{a} ∗ P_{air} + 2,1
$$

El índice de transmisividad de radiación difusa es estimó mediante $T_B$ para diferentes valores:

$$
T_D = 0.35-0.36T_B \text{  para  } T_B \geq 0.15 
$$
$\space$
$$
T_D = 0.18-0.82T_B \text{  para  } T_B \lt 0.15 
$$



In [ ]:
# El método numérico de Python exp() devuelve un resultado exponencial de x: e^x
# Para exp usar: math.exp()
a = 3
import math
math.exp(a) # 20.085536923187668

20.085536923187668

- Coeficientes de ponderación de la irradiancia exoatmosférica de la banda $\lambda$

$$
\alpha_s = \sum_{b=1}^{n}[\rho_{s,b} \times \omega_b]
$$

- $\rho_\lambda$ : Valor de la reflectancia de la banda $\lambda$ ✔️
- $\omega_\lambda$ : Coef. de ponderación de la irradiancia exoatmosférica de la banda $\lambda$ ✔️

##### Visualización de albedo


Datasets:
- [USGS Landsat 8 Level 2, Collection 2, Tier 1](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2) 
- [USGS Landsat 8 Surface Reflectance Tier 1 [deprecated]](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR)  

In [ ]:
# @markdown Función: `getAlbedo(image)`

def getAlbedo(image):
  """
  Obtener el albedo de una imagen mediante una calibración con 
  coeficientes de ponderación por banda (Da Silva et al., 2016)
  https://www.scielo.br/j/rbeaa/a/sX6cJjNXWMfHQ5p4h33B8Zz/?lang=en

  Argumentos:
      image (ee.Image) : Imagen satelital de Earth Engine.

  Retorna: 
      albedo (ee.Image) : Albedo de una imagen satelital. 
      
  """

  # Obtener nombres de las bandas de la imagen
  bandnames = image.bandNames().getInfo()[1:7]

  # Expresión para obtener el albedo
  albedo = image.expression(
      '(0.3*B2 + 0.277*B3 + 0.233*B4 + 0.143*B5 + 0.036*B6 + 0.012*B7)',
      {
          'B2' : image.select(bandnames[0]),
          'B3' : image.select(bandnames[1]),
          'B4' : image.select(bandnames[2]),
          'B5' : image.select(bandnames[3]),
          'B6' : image.select(bandnames[4]),
          'B7' : image.select(bandnames[5])
      }
  )

  return albedo

In [ ]:
# @markdown Paleta NDVI (Yuri SIG) 20 Colores: `yuri_ndvi_pal`
yuri_ndvi_pal = [
    'ffffff', 'fffcff', 'fff9ff', 'fff7ff', 'fff4ff', 'fff2ff', 'ffefff',
    'ffecff', 'ffeaff', 'ffe7ff', 'ffe5ff', 'ffe2ff', 'ffe0ff', 'ffddff',
    'ffdaff', 'ffd8ff', 'ffd5ff', 'ffd3ff', 'ffd0ff', 'ffceff', 'ffcbff',
    'ffc8ff', 'ffc6ff', 'ffc3ff', 'ffc1ff', 'ffbeff', 'ffbcff', 'ffb9ff',
    'ffb6ff', 'ffb4ff', 'ffb1ff', 'ffafff', 'ffacff', 'ffaaff', 'ffa7ff',
    'ffa4ff', 'ffa2ff', 'ff9fff', 'ff9dff', 'ff9aff', 'ff97ff', 'ff95ff',
    'ff92ff', 'ff90ff', 'ff8dff', 'ff8bff', 'ff88ff', 'ff85ff', 'ff83ff',
    'ff80ff', 'ff7eff', 'ff7bff', 'ff79ff', 'ff76ff', 'ff73ff', 'ff71ff',
    'ff6eff', 'ff6cff', 'ff69ff', 'ff67ff', 'ff64ff', 'ff61ff', 'ff5fff',
    'ff5cff', 'ff5aff', 'ff57ff', 'ff55ff', 'ff52ff', 'ff4fff', 'ff4dff',
    'ff4aff', 'ff48ff', 'ff45ff', 'ff42ff', 'ff40ff', 'ff3dff', 'ff3bff',
    'ff38ff', 'ff36ff', 'ff33ff', 'ff30ff', 'ff2eff', 'ff2bff', 'ff29ff',
    'ff26ff', 'ff24ff', 'ff21ff', 'ff1eff', 'ff1cff', 'ff19ff', 'ff17ff',
    'ff14ff', 'ff12ff', 'ff0fff', 'ff0cff', 'ff0aff', 'ff07ff', 'ff05ff',
    'ff02ff', 'ff00ff', 'ff00ff', 'ff0af4', 'ff15e9', 'ff1fdf', 'ff2ad4',
    'ff35c9', 'ff3fbf', 'ff4ab4', 'ff55aa', 'ff5f9f', 'ff6a94', 'ff748a',
    'ff7f7f', 'ff8a74', 'ff946a', 'ff9f5f', 'ffaa55', 'ffb44a', 'ffbf3f',
    'ffc935', 'ffd42a', 'ffdf1f', 'ffe915', 'fff40a', 'ffff00', 'ffff00',
    'fffb00', 'fff700', 'fff300', 'fff000', 'ffec00', 'ffe800', 'ffe400',
    'ffe100', 'ffdd00', 'ffd900', 'ffd500', 'ffd200', 'ffce00', 'ffca00',
    'ffc600', 'ffc300', 'ffbf00', 'ffbb00', 'ffb700', 'ffb400', 'ffb000',
    'ffac00', 'ffa800', 'ffa500', 'ffa500', 'f7a400', 'f0a300', 'e8a200',
    'e1a200', 'd9a100', 'd2a000', 'ca9f00', 'c39f00', 'bb9e00', 'b49d00',
    'ac9c00', 'a59c00', '9d9b00', '969a00', '8e9900', '879900', '7f9800',
    '789700', '709700', '699600', '619500', '5a9400', '529400', '4b9300',
    '439200', '349100', '2d9000', '258f00', '1e8e00', '168e00', '0f8d00',
    '078c00', '008c00', '008c00', '008700', '008300', '007f00', '007a00',
    '007600', '007200', '006e00', '006900', '006500', '006100', '005c00',
    '005800', '005400', '005000', '004c00'
]

In [ ]:
# @markdown Comparación entre L8_L2 y L8_SR: Albedo

roi = roiChongoyape

img_C1 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_010065_20200116').multiply(0.0001) 
img_C1_viz = {'min':0, 'max':0.3, 'gamma':1.4, 'bands':['B4','B3','B2']}
img_C1_albedo = getAlbedo(img_C1).clip(roi)

img_C2 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_010065_20200116').multiply(0.0000275).add(-0.2)
img_C2_viz = {'min':0, 'max':0.3, 'gamma':1.4, 'bands':['SR_B4','SR_B3','SR_B2']}
img_C2_albedo = getAlbedo(img_C2).clip(roi)

viz_params = {'min': 0.0, 'max':0.3, 'palette': yuri_ndvi_pal};

# Visualización
Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)
Map.centerObject(roi, 11) # Map.setCenter(lon, lat, zoom)
Map.addLayer(image_ID_C1, image_ID_C1_viz, name = 'L8 C1 Imagen')
Map.addLayer(img_C1_albedo, viz_params, name = 'L8 C1 Albedo')
Map.addLayer(image_ID_C2, image_ID_C2_viz, name = 'L8 C2 Imagen')
Map.addLayer(img_C2_albedo, viz_params, name = 'L8 C2 Albedo')
Map

Map(center=[-6.626857365812768, -79.46457298069963], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# @markdown Sentinel-2: Albedo

def verAlbedo(max):
  img_id = 'COPERNICUS/S2_SR/20200421T153621_20200421T154337_T17MPN'
  img = ee.Image(img_id).multiply(0.0001) 

  rgb_dict = {'min':0, 'max':0.3, 'bands':['B4','B3','B2']}
  albedo_dict = {'min': 0.0, 'max':max, 'palette': yuri_ndvi_pal};

  roi = roiChongoyape
  img_albedo = getAlbedo(image_ID_C1).clip(roi)

  # Visualización
  Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)
  Map.centerObject(roi, 11) # Map.setCenter(lon, lat, zoom)
  Map.addLayer(img, rgb_dict, 'Imagen Sentinel-2')
  Map.addLayer(img_albedo, viz_params, 'Albedo')
  return Map

interact(verAlbedo, max=(0, 1, 0.01));

interactive(children=(FloatSlider(value=0.0, description='max', max=1.0, step=0.01), Output()), _dom_classes=(…

**Fechas de campo: Setiembre a Diciembre**

Octubre:
- 20
- 22 
- 23
- 25
- 28 

Noviembre:
- 14 MSI FLIR H20T
- 17 MSI H20T
- 18 MsI FLIR H20T
- 20 H20T

##### Proyección de imagen

In [ ]:
image_ID_C1 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_010065_20200116').multiply(0.0001)

# Verificando proyección (Coordinate Reference System o CRS)
print('Projection, crs, and crs_transform', image_ID_C1.projection().getInfo()) # Sin getInfo es ee.Projection
print('Scale in meters:', image_ID_C1.projection().nominalScale().getInfo()) # 30m

Nótese la proyección `EPSG:32617` el cual pertenece a la zona 17N en UTM.

In [ ]:
# Reproyección a EPSG:32717 WGS 84 / UTM zone 17S
image_ID_C1_proy = image_ID_C1.reproject(crs="EPSG:32717", scale=30)
image_ID_C1_proy.projection().getInfo()

In [ ]:
# Visualización
image_ID_C1 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_010065_20200116') 
image_ID_esc_C1 = image_ID_C1.multiply(0.0001)
image_ID_esc_C1_reproy = image_ID_C1.multiply(0.0001).reproject(crs="EPSG:32717", scale=30)


dict_vis = {'Imagen L8 C1':image_ID_esc_C1.getMapId({'min':0, 'max':0.3, 'bands':['B4','B3','B2']}),
            'Imagen L8 C1 reproyectado':image_ID_esc_C1_reproy.getMapId({'min':0, 'max':0.3, 'bands':['B4','B3','B2']}),
            }

mapdisplay([-79.309, -6.846], dict_vis, zoom_start=9)

# mapdisplay([-79.309, -6.846], {'Albedo':albedo.clip(roiDep).getMapId(viz_params)},zoom_start=8)

#### Radiación de onda corta entrante / Incoming Solar Radiation $R_{S\downarrow}$ 

$R_{S\downarrow}$ : Radiación de onda corte entrante $[W/m²]$

Morse et al. (2000)

$$
R_{S\downarrow} = \frac{G_{sc} \cos\theta_{rel} \tau_{sw}}{d^2}
$$

Dónde:
-
- $G_{sc}$ : Constante solar $[1367 \space W/m²]$ ✔️
- $\theta$ : Ángulo zenital solar / Ángulo de incidencia solar $[rad]$ ✔️ 
- $d^2$ : Cuadrado de la distancia relativa Tierra-Sol $[m^{-1}]$ ✔️ (OJO: en 2018 se menciona el inverso del cuadrado de la distancia relativa de tierra al sol)
- $\tau_{sw}$ : Es la transmitancia en un sentido con condiciones de claridad / Transmitancia de la banda ancha $[W/m² \space K]$

In [ ]:
image_ID.getInfo()

In [ ]:
import math

ang_zenital = image_ID.get('SOLAR_ZENITH_ANGLE').getInfo()*math.pi/180
G_sc = 1367
dr = 1/image_ID.get('EARTH_SUN_DISTANCE').getInfo()

In [ ]:
1/0.983642

In [ ]:
# imagen = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_010065_20200116')
zenitAngle = image.get('SOLAR_ZENITH_ANGLE').getInfo()
zenitAngle

In [ ]:
R_s_entrante = 1367*cos(zenitAngle)*d_r*tau_sw

#### Radiación de onda larga saliente / Outgoing Long-Wave Radiation $R_{L\uparrow}$ 

Outgoing long-wave radiation, RL↑, emitted from the surface is
driven by surface temperature and surface emissivity. RL↑ is computed using the Stefan–Boltzmann equation

Morse et al., 2000.

$$
R_{L\downarrow} = \epsilon_0 * \sigma * T_{s}^{4}
$$

Dónde:
- $R_{L\downarrow}$ : Radiación de onda larga saliente $[W/m²]$
- $\epsilon_0$ : Emisividad del ancho de banda de la superficie
- $\sigma$ : Constante de Stefan-Boltzmann $(5.67*10^{-8}[W m^{-2}K^{-4}])$
- $T_s$ : Temperatura de brillo de superficie $[K]$

Para hallar la emisividad:

$$
\epsilon_0 = 0.95 + 0.01 * LAI \text{, cuando NDVI > 0 y LAI < 3} 
$$

Dónde:
- $\text{LAI}$ : es el índice de área foliar
- $\text{NDVI}$ : El índice de vegetación de diferencia normalizada

In [ ]:
R_l_saliente = emisividad*(5.67*10**-8)*temp_brillo**4

#### Radiación de onda larga entrante / Incoming Long-Wave Radiation $R_{L\downarrow}$

Incoming long-wave radiation is the downward thermal radiation
flux originating from the atmosphere W m−2 and is traditionally
computed using the Stefan–Boltzmann equation

Bastiaansen et al. 1998:

$$
R_{L\downarrow} = \epsilon_a \sigma T_{0 ref}^{4}
$$

Dónde:
- $\epsilon_a$ : Emisividad atmosférica efectiva
- $\sigma$ : Constante de Stefan-Boltzmann $(5.67*10^{-8} [W/m^{2}])$
- $T_{0 ref}$ : Temperatura en un punto de referencia con buen riego / near-surface air temperature $[K]$

> Ecuación empírica para $\epsilon_a$ de Bastiaanssen (1995) aplicado con coeficientes desarrollados por Allen et al. (2000) con datos recolectados de **alfalfa** en Idaho:
$$
\epsilon_a = 0.85(-\ln{\tau_{sw}})^{0.09}
$$

> The original coefficients
by Bastiaanssen (1995), derived for western Egypt, were
$$
\epsilon_a = 1.08 * (-\ln{\tau_{sw}})^{0.265}
$$

Dónde:
- $R_{L\downarrow}$ : Radiación de onda corta entrante
- $\tau_{sw}$ : Es la transmitancia en un sentido con condiciones de claridad



#### Radiación neta de la superficie $R_n$

### Flujo de calor del suelo / Soil Heat Flux $G$

> **Forma 1**
>
>Choudhury, Idso y Reginato (1987) o Allen, Pereira, Raes y Smith (1998)
>
>$$
G = 0.4 * \mathrm{e}^{-0.5* \text{LAI}} * R_n
$$
>
>Dónde:
>- $G$ : Flujo de calor del suelo $[W/m^{2}]$
>- $\text{LAI}$ : Índice de área foliar
>- $R_n$ : Flujo de radiación neta $[W/m^{2}]$

> **Forma 2**
>
> Despeje de la relación de calor almacenado en el suelo y vegetación debida a la conducción $R_n$, según Bastiaanssen (2000)
>
>$$
\frac{G}{R_n} = \frac{T_s}{\alpha}(0.0038\alpha + 0.0074\alpha^{2})(1-0.98*\text{NDVI}^4) 
$$
> Dónde:
> - $\frac{G}{R_n}$ : Relación del calor almacenado en suelo y vegetación
> - $T_s$ : Temperatura de brillo de superficie $[K]$
> - $\alpha$ : Albedo de superficie
> - $\text{NDVI}$ : Índice de vegetación de diferencia normalizada

#### Momento de resistencia aerodinámica (MRA) o $Z_{om}$

Chemin y Din Ahmad (2000), toman en cuenta las condiciones aerodinámicas de la vegetación la cual está en función de su altura y distribución espacial.

$$
Z_{om} = a + b*\text{NDVI}
$$

Dónde:

$$
a = \ln(0.02) - [b*(0.02)]
$$

$$
b = \frac{(\text{NDVI}_{max}-0.02)}{\ln(\frac{h_v}{7}-\ln(0.002)}
$$

- $\text{NDVI}$ : Índice de vegetación de diferencia normalizada
- $\text{NDVI}_{max}$ : Valor local máximo del $\text{NDVI}$
- $h_v$ : altura promedio de la vegetación cuando se tiene $\text{NDVI}_{max}$

#### Selección de pixel caliente y frío

Requisito para conocer el aporte de flujo de calor sensible $H$ en el modelo de balance, cada pixel con vegetación requerirá de conocer un gradiente de temperatura $\delta T$ en referencia a su condición aerodinámica a fin de estimar la densidad de su aporte, esto se logra a partir de la relación lineal que guardan las temperaturas entre dos superficies extremas con características distintas. 

Se han propuesto metodologías para la selección de estos puntos extremos denominándose **pixel frío** y **pixel caliente**, donde: 
- un **pixel frío** representa una superficie con vegetación densa y húmeda, mientras 
- un **pixel caliente** representa una superficie seca sin vegetación. 

Según el método de Olmedo et al. (2016), la selección de estos dos parámetros puede llevarse sobre el área de estudio y localizarlos mediante el cumplimiento de una serie de condiciones sobre otros parámetros conocidos como se muestra a continuación:

$\space$

$$
\text{pixel frio} = 
\begin{cases}
  \begin{gather*}
    3 \le \text{LAI} \le 6 \\
    0.18 \le \alpha \le 0.25 \\
    0.03 \le Z_{0m} \le 0.08 \\
    \text{NDVI} \ge \text{NDVI}_{max} - 0.15 \\
    T_s < T_{max} - 5 
  \end{gather*}
\end{cases}
$$

$\space$

$$
\text{pixel caliente} = 
\begin{cases}
  \begin{gather*}
    0.13 \le \alpha \le 0.15 \\
    Z_{0m} \le 0.005 \\
    0.1 \le \text{NDVI} \le 0.28 \\
    T_s > T_{max} - 5
  \end{gather*}
\end{cases}
$$

Dónde:
- $\alpha$ : Es el albedo de superficie
- $\text{LAI}$ : Índice de área foliar
- $Z_{0m}$ : Momento de resistencia aerodinámica o MRA
- $\text{NDVI}$ : Índice de vegetación de diferencia normalizada y máximo $\text{NDVI}_{max}$
- $T_s$ : Temperatura de superficie y máximo valor $T_{max}$


### Flujo de calor sensible / Sensible Heat Flux $H$

Bastiaanssen et al. (1998)

$$
H = \frac{\rho_a c_p \delta{T_a}}{r_{ah}}
$$

$$
H = \rho_{air} C_p \frac{dT}{r_{ah}}
$$

Dónde:
- $H$ : Flujo de calor sensible $[W/m^{2}]$
- $\rho_a$ : Densidad del aire $[kg/m^{3}]$
- $c_p$ : Calor específico del aire igual a $[1004 \space J/kg/K]$
- $\delta{T_a}$ : Gradiente de temperatura $[°C]$
- $r_{ah}$ : Resistencia aerodinámica al transporte de calor $[s/m]$ 

### $\lambda{ET}$ y $ET$ instantánea

### Evapotranspiración de cultivo $ET_c$ por día

La $ET_c$ diaria se puede calcular mediante la siguiente ecuación:

$$
ET_c[\frac{mm}{d}] = C_{rad} * \Lambda * ET_{24}
$$

In [ ]:
# savi, iaf, pasar defrente del ndvi al iaf, estacion climatica en tinajones, CHONGOYAPE SOLAMENTE
# hasta KC a partir de diversas ecuaciones de estacion meteorologica

# Bibliografía

**METRIC**

Búsqueda en Connected Papers
- [ConectedPapers - 2007: Satellite-Based Energy Balance for Mapping Evapotranspiration with Internalized Calibration (METRIC) — Model](https://www.connectedpapers.com/main/817edad756d41499da26498e71c85afaa884383a/SatelliteBased-Energy-Balance-for-Mapping-Evapotranspiration-with-Internalized-Calibration-METRICModel/graph)

Artículos científicos:
- Allen, R. G., Tasumi, M., & Trezza, R. (2007). Satellite-based energy balance for mapping evapotranspiration with internalized calibration (METRIC)—Model. Journal of irrigation and drainage engineering, 133(4), 380-394. doi:[10.1061/(ASCE)0733-9437(2007)133:4(380)](http://cursosihlla.bdh.org.ar/ET_UNCu_Cursos/Curso_2019/1_Lectura_recomendada/2007_Allen_METRIC_Satellite-Based_Energy_Balance_for_Mapping_Evapotr.pdf)
- Kilic, A., Allen, R., Huntington, J., y Ozturk, D. (2018). Earth Engine Evapotranspiration Flux v.0.10.4. Recuperado el 1 de julio de 2018, a partir de http://eeflux-level1.appspot.com/

Tesis:
- Vázquez, B. A. (2019). Estimación de la evapotranspiración de cultivo de maíz bajo riego mediante percepción remota (Tesis de maestría, Instituto Mexicano de Tecnología del Agua). Recuperada de http://repositorio.imta.mx/handle/20.500.12013/2065

- Príncipe, R. E. (2018). Estimación de la evapotranspiración en los cultivos alrededor del observatorio de Huancayo mediante sensoramiento remoto (Tesis de pregrado, Universidad Nacional Mayor de San Marcos). Recuperada de https://repositorio.igp.gob.pe/handle/20.500.12816/4631

<img src='https://mms.businesswire.com/media/20180706005099/en/666704/4/INTECHOPEN_LOGO_RED_RGB.jpg?download=1' width=4%>**IntechOpen**

- Irmak, A., Allen, R. G., Kjaersgaard, J., Huntington, J., Kamble, B., Trezza, R., & Ratcliffe, I. (2012). Operational remote sensing of ET and challenges. Evapotranspiration—Remote Sensing and Modeling, 467-492. doi:[10.5772/25174](https://www.intechopen.com/chapters/26117)
- Trezza, R., Allen, R. G., Kilic, A., Ratcliffe, I., & Tasumi, M. (2018). Influence of Landsat revisit frequency on time-integration of evapotranspiration for agricultural water management. Advanced Evapotranspiration Methods and Applications. doi: [10.5772/intechopen.80946](https://www.intechopen.com/chapters/64446)

**Revisión de métodos que usan Sensoramiento Remoto**
- [Connected papers](https://www.connectedpapers.com/main/8886bf69220b2ec6f88c477d9e3ce9da5587e396/Estimating-Land-Surface-Evaporation%3A-A-Review-of-Methods-Using-Remotely-Sensed-Surface-Temperature-Data/graph)
- Kalma, J. D., McVicar, T. R., & McCabe, M. F. (2008). Estimating Land Surface Evaporation: A Review of Methods Using Remotely Sensed Surface Temperature Data. Surveys in Geophysics, 29(4), 421-469. https://doi.org/10.1007/s10712-008-9037-z
- Khand, K., Taghvaeian, S., Gowda, P., & Paul, G. (2019). A Modeling Framework for Deriving Daily Time Series of Evapotranspiration Maps Using a Surface Energy Balance Model. Remote Sensing, 11(5), 508. https://doi.org/10.3390/rs11050508


**Búsquedas de METRIC Evapotranspiration**

- [Búsqueda en *ScienceDirect*: METRIC Evapotranspiration](https://www.sciencedirect.com/search?qs=metric%20evapotranspiration)
- [Búsqueda en *MDPI*: METRIC Evapotranspiration](https://www.mdpi.com/search?q=metric+evapotranspiration)

**Investigadores de ETP**

- [Richard Allen: University of Idaho](https://www.uidaho.edu/cals/soil-and-water-systems/our-people/richard-allen)
- [Ayse Kilic: University of Nebraska - Lincoln](https://engineering.unl.edu/civil/ayse-kilic/)


**Por revisar**

Artículos cientificos:
- De la Fuente-Sáiz, D., Ortega-Farías, S., Fonseca, D., Ortega-Salazar, S., Kilic, A., & Allen, R. (2017). Calibration of METRIC Model to Estimate Energy Balance over a Drip-Irrigated Apple Orchard. Remote Sensing, 9(7), 670. https://doi.org/10.3390/rs9070670
- Bhattarai, N., & Liu, T. (2019). LandMOD ET mapper: A new matlab-based graphical user interface (GUI) for automated implementation of SEBAL and METRIC models in thermal imagery. Environmental Modelling & Software, 118, 76-82. https://doi.org/10.1016/j.envsoft.2019.04.007

Libro:
- [A Review of Surface Energy Balance Models for Estimating 
Actual Evapotranspiration with Remote Sensing at High 
Spatiotemporal Resolution over Large Extents](https://pubs.usgs.gov/sir/2017/5087/sir20175087.pdf)

Páginas web:
- Lista de principales símbolos y acrónimos para evapotranspiración: [FAO](http://www.fao.org/3/X0490E/x0490e03.htm#list%20of%20principal%20symbols%20and%20acronyms)
- Conversión de valores de ND a radiancia: [youtube](https://www.youtube.com/watch?v=kRN1ekslBH0&list=PLkMtpdwUewHG5JvNoZbOL7-c6_EXoS_UI)
- [Applied Remote Sensing Training ARSET: NASA Evapotranspiration Data Products and Applications](http://www.cazalac.org/mwar_lac/fileadmin/imagenes2/Remote_Sensing/S4P1_light.pdf)

**Recursos de $\LaTeX$ empleados**
- [LaTex mathematics for equations](https://en.wikibooks.org/wiki/LaTeX/Mathematics)
- [Overleaf - Aligning equations with amsmath](https://www.overleaf.com/learn/latex/Aligning_equations_with_amsmath)
- [Wumbo - Mathematical Symbols](https://wumbo.net/symbols/)



In [ ]:
# @markdown Typical Landsat Collection 2 Product Generation
%%html
<p align='center'>
  <img src='https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/thumbnails/image/Landsat%20Collection%202%20timeline-web.jpg' width='90%'>
</p>